In [1]:
import torch
import torch.autograd as autograd         # computation graph
from torch import Tensor                  # tensor node in the computation graph
import torch.nn as nn                     # neural networks
import torch.optim as optim               # optimizers e.g. gradient descent, ADAM, etc.

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.ticker
from torch.nn.parameter import Parameter

import numpy as np
import time
from pyDOE import lhs         #Latin Hypercube Sampling
import scipy.io

from smt.sampling_methods import LHS
from scipy.io import savemat

#Set default dtype to float32
torch.set_default_dtype(torch.float)

#PyTorch random number generator
torch.manual_seed(1234)

# Random number generators in other libraries
np.random.seed(1234)

# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

print(device)

if device == 'cuda': 
    print(torch.cuda.get_device_name())

    

cuda:0


In [2]:
def true_1D_1(x): #True function for 1D_1 dy2/dx2 + dy/dx - 6y = 0; BC1: y(0)=2; BC2: dy/dx at (x=0) = -1;
    y = np.exp(2*x) + np.exp(-3*x)
    return y/10
    

In [3]:
loss_thresh = 0.005
label = "1D_SODE_rowdy_scaled"

x = np.linspace(0,2,100).reshape(-1,1)

bc1_x = x[0].reshape(-1,1)
bc1_y = true_1D_1(x[0]).reshape(-1,1)
x_bc1_train = torch.from_numpy(bc1_x).float().to(device)
y_bc1_train = torch.from_numpy(bc1_y).float().to(device)
    

bc2_x = x[0].reshape(-1,1)
x_bc2_train = torch.from_numpy(bc2_x).float().to(device)
bc2_val = torch.tensor(-1.0/10.0,device=device)
bc2_val = bc2_val.view(1,1)

x_test = x.reshape(-1,1)
x_test_tensor = torch.from_numpy(x_test).float().to(device)
y_true = true_1D_1(x_test)
y_true_norm = np.linalg.norm(y_true,2)

# Domain bounds
lb = np.array(x[0]) 
ub = np.array(x[-1]) 

In [4]:
def colloc_pts(N_f,seed):
    #Collocation Points
    # Latin Hypercube sampling for collocation points 
    # N_f sets of tuples(x,y)
    x01 = np.array([[0.0, 1.0]])
    sampling = LHS(xlimits=x01,random_state =seed)
    
    x_coll_train = lb + (ub-lb)*sampling(N_f)
    x_coll_train = np.vstack((x_coll_train, bc1_x.reshape(-1,1))) # append training points to collocation points 

    return x_coll_train

In [5]:
class Sequentialmodel(nn.Module):
    
    def __init__(self,layers,n_val,rowdy_terms):
        super().__init__() #call __init__ from parent class 
              
    
        self.activation = nn.Tanh()
        self.loss_function = nn.MSELoss(reduction ='mean')
        
        'Initialise neural network as a list using nn.Modulelist'  
        self.linears = nn.ModuleList([nn.Linear(layers[i], layers[i+1]) for i in range(len(layers)-1)])
        
        for i in range(len(layers)-1):
            nn.init.xavier_normal_(self.linears[i].weight.data, gain=1.0)
            # set biases to zero
            nn.init.zeros_(self.linears[i].bias.data) 
        
        
        self.omega1 = Parameter(torch.ones((len(layers)-2,1))) 
        
        
        self.alpha = Parameter(torch.zeros(rowdy_terms,len(layers)-2))
        self.omega = Parameter(0.1*torch.ones(rowdy_terms,len(layers)-2))
        
        self.n = torch.tensor(n_val)
    
    'forward pass'
    def forward(self,x):
        if torch.is_tensor(x) != True:         
            x = torch.from_numpy(x)                
        
        u_b = torch.from_numpy(ub).float().to(device)
        l_b = torch.from_numpy(lb).float().to(device)
                      
        #preprocessing input 
        x = (x - l_b)/(u_b - l_b) #feature scaling
        
        #convert to float
        a = x.float()
        
        for i in range(len(layers)-2):
            z = self.linears[i](a)
            a = self.activation(self.omega1[i,0]*z)
            for j in range(rowdy_terms):
                a = a + self.alpha[j,i]*self.n*torch.sin((j+1)*self.n*self.omega[j,i]*z)
            
        a = self.linears[-1](a) 
         
        return a
                        
    def loss_BC1(self,x,y):
                
        loss_bc1 = self.loss_function(self.forward(x), y)
                
        return loss_bc1
    
    def loss_BC2(self,x_bc2,bc2_val):
        g = x_bc2.clone()             
        g.requires_grad = True
        y = self.forward(g)    
            
        y_x = autograd.grad(y,g,torch.ones([x_bc2.shape[0], 1]).to(device), retain_graph=True, create_graph=True,allow_unused = True)[0]
        
        dy_dx = y_x[:,[0]]
        
        bc2 = dy_dx
        
        loss_bc2= self.loss_function(bc2,bc2_val)

        return loss_bc2
    
    def loss_PDE(self, x_coll,f_hat):
             
        g = x_coll.clone()             
        g.requires_grad = True
  
        y = self.forward(g) 

        y_x = autograd.grad(y,g,torch.ones([x_coll.shape[0], 1]).to(device), retain_graph=True, create_graph=True,allow_unused = True)[0]
        y_xx = autograd.grad(y_x,g,torch.ones(x_coll.shape).to(device), create_graph=True,allow_unused = True)[0]

        dy_dx = y_x[:,[0]]
        
        dy2_d2x = y_xx[:,[0]]
        
        f = dy2_d2x + dy_dx - 6*y
        
        loss_f = self.loss_function(f,f_hat)
                
        return loss_f
    
    
    def loss(self,x_bc1,y_bc1,x_bc2,bc2_val,x_coll,f_hat):

        loss_bc1 = self.loss_BC1(x_bc1,y_bc1)
        loss_bc2 = self.loss_BC2(x_bc2,bc2_val)
        loss_f = self.loss_PDE(x_coll,f_hat)
        
        loss_val = loss_bc1 + loss_bc2 + loss_f
        
        return loss_val
          
    'test neural network'
    
    def test(self):
        y_pred = self.forward(x_test_tensor)
        y_pred = y_pred.cpu().detach().numpy()

        return y_pred
    
    def test_loss(self):
        y_pred = self.test()
        
        test_mse = np.mean(np.square(y_pred.reshape(-1,1) - y_true.reshape(-1,1)))
        test_re = np.linalg.norm(y_pred.reshape(-1,1) - y_true.reshape(-1,1),2)/y_true_norm
        
        return test_mse, test_re

In [6]:
def train_step(x_coll,f_hat):
    def closure():
        optimizer.zero_grad()
        loss = PINN.loss(x_bc1_train,y_bc1_train,x_bc2_train,bc2_val,x_coll,f_hat)
        loss.backward()
        
        return loss

    optimizer.step(closure)

In [7]:
def data_update(loss_np):
    train_loss.append(loss_np)
    alpha_val.append(PINN.alpha.cpu().detach().numpy())
    omega_val.append(PINN.omega.cpu().detach().numpy())
    test_mse, test_re = PINN.test_loss()
    test_mse_loss.append(test_mse)
    test_re_loss.append(test_re)

In [8]:
def train_model(max_iter,rep):
    print(rep) 
    torch.manual_seed(rep*123)
    start_time = time.time()
    thresh_flag = 0

    for i in range(max_iter):
        x_coll = torch.from_numpy(colloc_pts(N_f,i*11)).float().to(device)
        f_hat = torch.zeros(x_coll.shape[0],1).to(device)
        train_step(x_coll,f_hat)
        
        loss_np = PINN.loss(x_bc1_train,y_bc1_train,x_bc2_train,bc2_val,x_coll,f_hat).cpu().detach().numpy()
        if(thresh_flag == 0):
            if(loss_np < loss_thresh):
                time_threshold[rep] = time.time() - start_time
                epoch_threshold[rep] = i+1            
                thresh_flag = 1       
        data_update(loss_np)
        print(i,"Train Loss",train_loss[-1],"Test MSE",test_mse_loss[-1],"Test RE",test_re_loss[-1])
    
    elapsed_time[rep] = time.time() - start_time
    print('Training time: %.2f' % (elapsed_time[rep]))

In [14]:
max_reps = 10
max_iter = 100


N_f = 1000

train_loss_full = []
test_mse_full = []
test_re_full = []
alpha_full = []
omega_full = []
elapsed_time= np.zeros((max_reps,1))

time_threshold = np.empty((max_reps,1))
time_threshold[:] = np.nan
epoch_threshold = max_iter*np.ones((max_reps,1))

n_val = 8.0
rowdy_terms = 2

for reps in range(max_reps):

    train_loss = []
    test_mse_loss = []
    test_re_loss =[]
    alpha_val = []
    omega_val = []

    'Generate Training data'
    torch.manual_seed(reps*36)
     #Total number of collocation points 



    layers = np.array([1,50,50,50,50,50,50,50,50,50,1]) #9 hidden layers
    PINN = Sequentialmodel(layers,n_val,rowdy_terms)
    PINN.to(device)

    'Neural Network Summary'
    print(PINN)

    params = list(PINN.parameters())

    optimizer = torch.optim.LBFGS(PINN.parameters(), lr=0.05, 
                              max_iter = 10, 
                              max_eval = 15, 
                              tolerance_grad = 1e-9, 
                              tolerance_change = 1e-9, 
                              history_size = 100, 
                              line_search_fn = 'strong_wolfe')


    train_model(max_iter,reps)


    torch.save(PINN.state_dict(),label+'_'+str(reps)+'.pt')
    train_loss_full.append(train_loss)
    test_mse_full.append(test_mse_loss)
    test_re_full.append(test_re_loss)
    alpha_full.append(alpha_val)
    omega_full.append(omega_val)  

    print('Training time: %.2f' % (elapsed_time[reps]))

mdic = {"train_loss": train_loss_full,"test_mse_loss": test_mse_full, "test_re_loss": test_re_full, "Time": elapsed_time, "alpha": alpha_full, "omega": omega_full, "label": label,"Thresh Time": time_threshold,"Thresh epoch": epoch_threshold}
savemat(label+'.mat', mdic)

Sequentialmodel(
  (activation): Tanh()
  (loss_function): MSELoss()
  (linears): ModuleList(
    (0): Linear(in_features=1, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features=50, bias=True)
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): Linear(in_features=50, out_features=50, bias=True)
    (4): Linear(in_features=50, out_features=50, bias=True)
    (5): Linear(in_features=50, out_features=50, bias=True)
    (6): Linear(in_features=50, out_features=50, bias=True)
    (7): Linear(in_features=50, out_features=50, bias=True)
    (8): Linear(in_features=50, out_features=50, bias=True)
    (9): Linear(in_features=50, out_features=1, bias=True)
  )
)
0
0 Train Loss 0.054414906 Test MSE 3.926339730206851 Test RE 1.0100817393674
1 Train Loss 0.04388325 Test MSE 3.878048952493898 Test RE 1.0038509306671681
2 Train Loss 0.043418463 Test MSE 3.8663409398576674 Test RE 1.0023344485737864
3 Train Loss 0.042138517 Test MSE 3.84793099100453 Test RE 0.99994

91 Train Loss 0.00013808659 Test MSE 0.0005037080773810342 Test RE 0.011440685314133468
92 Train Loss 0.00013808659 Test MSE 0.0005037080773810342 Test RE 0.011440685314133468
93 Train Loss 0.00013808659 Test MSE 0.0005037080773810342 Test RE 0.011440685314133468
94 Train Loss 0.00013808659 Test MSE 0.0005037080773810342 Test RE 0.011440685314133468
95 Train Loss 0.00013808659 Test MSE 0.0005037080773810342 Test RE 0.011440685314133468
96 Train Loss 0.00013808659 Test MSE 0.0005037080773810342 Test RE 0.011440685314133468
97 Train Loss 0.00013808659 Test MSE 0.0005037080773810342 Test RE 0.011440685314133468
98 Train Loss 0.00013808659 Test MSE 0.0005037080773810342 Test RE 0.011440685314133468
99 Train Loss 0.00013808659 Test MSE 0.0005037080773810342 Test RE 0.011440685314133468
Training time: 88.43
Training time: 88.43
Sequentialmodel(
  (activation): Tanh()
  (loss_function): MSELoss()
  (linears): ModuleList(
    (0): Linear(in_features=1, out_features=50, bias=True)
    (1): Line

81 Train Loss 0.0001450754 Test MSE 0.0003277435090112609 Test RE 0.009228468456959916
82 Train Loss 0.00013908376 Test MSE 0.00122645680357518 Test RE 0.017852075335976807
83 Train Loss 0.00012793017 Test MSE 0.0026823083068836656 Test RE 0.02640079043820448
84 Train Loss 0.00011906495 Test MSE 0.002240543312768054 Test RE 0.02412899616784861
85 Train Loss 9.9791e-05 Test MSE 0.0006342456288733528 Test RE 0.012837821236155027
86 Train Loss 7.899649e-05 Test MSE 2.8244001427909795e-05 Test RE 0.0027091041085513454
87 Train Loss 6.145091e-05 Test MSE 7.648329587301814e-05 Test RE 0.004458059697970892
88 Train Loss 5.1790154e-05 Test MSE 0.00010584673691833029 Test RE 0.005244466547050337
89 Train Loss 4.632508e-05 Test MSE 3.224753458140547e-05 Test RE 0.0028947485236357415
90 Train Loss 4.0196872e-05 Test MSE 1.263850598721586e-06 Test RE 0.0005730736655522068
91 Train Loss 3.4289747e-05 Test MSE 8.399503173053942e-06 Test RE 0.0014773703603809456
92 Train Loss 2.4963512e-05 Test MSE 3

73 Train Loss 0.022955414 Test MSE 3.6573876067174593 Test RE 0.9748730777071511
74 Train Loss 0.022922779 Test MSE 3.6436875849168855 Test RE 0.9730455009609834
75 Train Loss 0.022911377 Test MSE 3.635869042372995 Test RE 0.9720009707956467
76 Train Loss 0.022837462 Test MSE 3.605494604656701 Test RE 0.9679323556798443
77 Train Loss 0.022659462 Test MSE 3.5836877377241065 Test RE 0.9650007766381855
78 Train Loss 0.022582117 Test MSE 3.5777524074044442 Test RE 0.9642013247500607
79 Train Loss 0.022441221 Test MSE 3.540675634616858 Test RE 0.9591922336139025
80 Train Loss 0.02234691 Test MSE 3.486422880298605 Test RE 0.9518151519514056
81 Train Loss 0.02226748 Test MSE 3.467941681267236 Test RE 0.949289058660087
82 Train Loss 0.022226542 Test MSE 3.4774588558159887 Test RE 0.95059074717503
83 Train Loss 0.022171997 Test MSE 3.4728958478844323 Test RE 0.9499668753864455
84 Train Loss 0.022147717 Test MSE 3.4790876941437254 Test RE 0.9508133489996975
85 Train Loss 0.022140384 Test MSE 3.4

66 Train Loss 0.02315776 Test MSE 3.6985671649766187 Test RE 0.9803459012342857
67 Train Loss 0.02315776 Test MSE 3.6985671649766187 Test RE 0.9803459012342857
68 Train Loss 0.02315776 Test MSE 3.6985671649766187 Test RE 0.9803459012342857
69 Train Loss 0.02315776 Test MSE 3.6985671649766187 Test RE 0.9803459012342857
70 Train Loss 0.02315776 Test MSE 3.6985671649766187 Test RE 0.9803459012342857
71 Train Loss 0.02315776 Test MSE 3.6985671649766187 Test RE 0.9803459012342857
72 Train Loss 0.02315776 Test MSE 3.6985671649766187 Test RE 0.9803459012342857
73 Train Loss 0.02315776 Test MSE 3.6985671649766187 Test RE 0.9803459012342857
74 Train Loss 0.02315776 Test MSE 3.6985671649766187 Test RE 0.9803459012342857
75 Train Loss 0.02315776 Test MSE 3.6985671649766187 Test RE 0.9803459012342857
76 Train Loss 0.02315776 Test MSE 3.6985671649766187 Test RE 0.9803459012342857
77 Train Loss 0.02315776 Test MSE 3.6985671649766187 Test RE 0.9803459012342857
78 Train Loss 0.02315776 Test MSE 3.6985

...

53 Train Loss 0.0049341186 Test MSE 0.5099972716670432 Test RE 0.3640378232940272
54 Train Loss 0.004378705 Test MSE 0.5085128694579685 Test RE 0.363507651506885
55 Train Loss 0.004215044 Test MSE 0.5401672223966215 Test RE 0.37465082757400964
56 Train Loss 0.0038749224 Test MSE 0.5318575742687415 Test RE 0.3717579432709376
57 Train Loss 0.0036902092 Test MSE 0.49350152760386273 Test RE 0.3581020714445177
58 Train Loss 0.0034679489 Test MSE 0.4385888668600807 Test RE 0.3375914074473453
59 Train Loss 0.0030818228 Test MSE 0.3783248685728377 Test RE 0.31354149694421163
60 Train Loss 0.00275786 Test MSE 0.3170097406093194 Test RE 0.2870112273872836
61 Train Loss 0.0026068764 Test MSE 0.28871672367914514 Test RE 0.273904112465382
62 Train Loss 0.0025532139 Test MSE 0.26832152112055196 Test RE 0.26405253192805184
63 Train Loss 0.0024820988 Test MSE 0.2409277998043416 Test RE 0.2502107932863735
64 Train Loss 0.0023093289 Test MSE 0.2041400878027963 Test RE 0.2303173903993907
65 Train Loss 0.

43 Train Loss 0.0059292167 Test MSE 0.7660224455506913 Test RE 0.44615249926047224
44 Train Loss 0.0054232404 Test MSE 0.6215141745904902 Test RE 0.40187232683955815
45 Train Loss 0.005056534 Test MSE 0.577744087727566 Test RE 0.387463089861588
46 Train Loss 0.004603532 Test MSE 0.5183238043366495 Test RE 0.36699754560264175
47 Train Loss 0.0044585806 Test MSE 0.481209284900807 Test RE 0.3536141065230207
48 Train Loss 0.0043001324 Test MSE 0.41253799612468955 Test RE 0.3274119752371264
49 Train Loss 0.004152066 Test MSE 0.3684372414297013 Test RE 0.3094171228853416
50 Train Loss 0.0038072367 Test MSE 0.3490935930179595 Test RE 0.3011851268066008
51 Train Loss 0.003574609 Test MSE 0.3688552630171094 Test RE 0.30959260239124675
52 Train Loss 0.003505102 Test MSE 0.4033288593123423 Test RE 0.3237369207580184
53 Train Loss 0.003318795 Test MSE 0.4209936587694035 Test RE 0.33075038598804246
54 Train Loss 0.0030976315 Test MSE 0.37240055364306457 Test RE 0.31107688504396697
55 Train Loss 0.0

34 Train Loss 0.023398886 Test MSE 3.738275459108865 Test RE 0.985594411693153
35 Train Loss 0.023396768 Test MSE 3.738718007921166 Test RE 0.9856527488506124
36 Train Loss 0.023394344 Test MSE 3.7387594925287204 Test RE 0.9856582172095469
37 Train Loss 0.02338966 Test MSE 3.7378614653661186 Test RE 0.9855398355629039
38 Train Loss 0.023375968 Test MSE 3.7290517798155003 Test RE 0.9843777515816893
39 Train Loss 0.0233604 Test MSE 3.714986784867023 Test RE 0.9825195918174119
40 Train Loss 0.023315052 Test MSE 3.7009964738472134 Test RE 0.9806678059002225
41 Train Loss 0.023257017 Test MSE 3.703418184047438 Test RE 0.9809885985401878
42 Train Loss 0.023156106 Test MSE 3.6965191434253892 Test RE 0.9800744383321335
43 Train Loss 0.023119627 Test MSE 3.6833759821755847 Test RE 0.9783305346367528
44 Train Loss 0.022943785 Test MSE 3.618322265511679 Test RE 0.9696526860698154
45 Train Loss 0.022592144 Test MSE 3.555127474021522 Test RE 0.9611477891702198
46 Train Loss 0.02206273 Test MSE 3.42

27 Train Loss 0.022387372 Test MSE 3.5828233996372414 Test RE 0.9648843969080166
28 Train Loss 0.022341736 Test MSE 3.574980488445489 Test RE 0.963827737438094
29 Train Loss 0.02220075 Test MSE 3.5335884583619084 Test RE 0.9582317715000486
30 Train Loss 0.021907806 Test MSE 3.503289207091226 Test RE 0.9541146803001241
31 Train Loss 0.020817373 Test MSE 3.270825634569768 Test RE 0.9219158442740453
32 Train Loss 0.02052193 Test MSE 3.239400288652434 Test RE 0.9174763767177022
33 Train Loss 0.020020934 Test MSE 3.1245914566975 Test RE 0.9010713950469104
34 Train Loss 0.01903764 Test MSE 2.9619290590256333 Test RE 0.8773035945913333
35 Train Loss 0.018506583 Test MSE 2.902289984878842 Test RE 0.8684263333514858
36 Train Loss 0.017922174 Test MSE 2.7576157451556265 Test RE 0.8465048633738753
37 Train Loss 0.01742142 Test MSE 2.631442608083112 Test RE 0.8269124557956857
38 Train Loss 0.017111335 Test MSE 2.5522336202287805 Test RE 0.8143719286863826
39 Train Loss 0.016866794 Test MSE 2.50107

18 Train Loss 0.023828397 Test MSE 3.8371847920367927 Test RE 0.9985479854031775
19 Train Loss 0.023828397 Test MSE 3.8371847920367927 Test RE 0.9985479854031775
20 Train Loss 0.023828397 Test MSE 3.8371847920367927 Test RE 0.9985479854031775
21 Train Loss 0.023828397 Test MSE 3.8371847920367927 Test RE 0.9985479854031775
22 Train Loss 0.023828397 Test MSE 3.837187541830998 Test RE 0.99854834319165
23 Train Loss 0.023828322 Test MSE 3.837197149182228 Test RE 0.9985495932476538
24 Train Loss 0.023828322 Test MSE 3.837197149182228 Test RE 0.9985495932476538
25 Train Loss 0.023828322 Test MSE 3.837197149182228 Test RE 0.9985495932476538
26 Train Loss 0.023828322 Test MSE 3.837197149182228 Test RE 0.9985495932476538
27 Train Loss 0.023828322 Test MSE 3.837197149182228 Test RE 0.9985495932476538
28 Train Loss 0.023828322 Test MSE 3.837197149182228 Test RE 0.9985495932476538
29 Train Loss 0.023828322 Test MSE 3.837197149182228 Test RE 0.9985495932476538
30 Train Loss 0.0238283 Test MSE 3.837

In [ ]:
lrnr_tune[tune_reps,2]

In [ ]:
import scipy.io as sio

In [ ]:
for tune_reps in range(75):
    label = "1D_SODE_rowdy_tune"+str(tune_reps)+".mat" #WRONGLY SAVED AS STAN - DOESN'T MATTER
    data = sio.loadmat(label)
    re = np.array(data["test_re_loss"])
    print(np.mean(re[:,-1]))

In [15]:
a = 0
for i in range(10):
    a = a + test_re_full[i][-1]
print(a/10)

0.44299637617173226
